In [16]:
import pandas as pd
import numpy as np

projection_keep = pd.read_csv('/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_agp3k/wgs/16.bbj_projection_sample_keep/CTEPH_AGP3K.bbj_projection.subset.keep.txt', 
                              header=0, sep='\t', dtype=str)

kinship_remove = pd.read_csv('/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_agp3k/wgs/17.kinship_sample_remove/kinship_check.to_remove.txt',
                              header=None, dtype=str)

projection_keep_filtered = projection_keep[~projection_keep['IID'].isin(kinship_remove[0])]
projection_keep_filtered.to_csv('sample_keep.txt',sep='\t', index=False, header=True)

In [17]:
import subprocess

def run_command(command):
    """Run a shell command (tuple/list or string) and return the output."""
    if isinstance(command, (tuple, list)):
        result = subprocess.run(command, capture_output=True, text=True)
    else:
        result = subprocess.run(command, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        raise RuntimeError(f"Command failed:\n{command}\n{result.stderr}")
    return result.stdout.strip()

bed_prefix = "/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_agp3k/wgs/14.hwe_qc/cteph_agp3k.s_qc.gt_qc.v_qc.hwe"
out_prefix = "cteph_agp3k.s_qc.gt_qc.v_qc.hwe.sample_keep"

cmd_sample_keep = [
    "/home/b/b37974/plink2",
    "--bfile", bed_prefix,
    "--keep", "sample_keep.txt",
    "--make-bed",
    "--out", out_prefix, 
    "--threads", "8"
]

# run_command(cmd_sample_keep)

In [18]:
fam_df = pd.read_csv(f'{out_prefix}.fam', sep='\t', header=None, dtype=str)
sex_df = fam_df[[0, 1, 4]].copy()
sex_df.columns = ['#FID', 'IID', 'SEX']

pheno_df = fam_df[[0, 1, 5]].copy()
pheno_df.columns = ['#FID', 'IID', 'PHENO1']
pheno_df.to_csv('pheno.txt', sep='\t', index=False, header=True)


In [20]:
sscore = pd.read_csv("/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_agp3k/bbj_projection/03.bbj_projection/cteph_agp3k.bbj.projection.sscore", 
                     header=0, sep='\t')

sscore_filtered = sscore[sscore['IID'].isin(projection_keep_filtered['IID'])].drop(sscore.columns[[2, 3, 4]], axis=1)
covariance_df = pd.merge(sex_df, sscore_filtered, on=['#FID', 'IID'], how='left')
covariance_df.to_csv('covariance.txt', sep='\t', index=False, header=True)
